## 모델 훑어 보기


문장 생성:
  이전 단어어들(**컨텍스트**)가 주어졌을 때 다음 단어로 어떤 단어가 오는지 분류하기

#### 모델의 구조
BERT(트랜스포머 인코더): 
- 프리트레인 태스크: 빈칸 맞히기
- 파인튜닝: 각 다운스림 태스크

GPT(트랜스포머 디코더):
- 프리트레인 태스크: 다음 단어 맞히기
- 파인튜닝: 다음 단어 맞히기

## 모델 학습하기

#### 설정하기

In [ ]:
# TPU 사용시 
#!pip install cloud-tpu-client==0.10 https://storage.googleapis.com/tpu-pytorch/wheels/torch_xla-1.9-cp37-cp37m-linux_x86_64.whl

In [1]:
!pip install ratsnlp

from google.colab import drive
drive.mount('/gdrive', force_remount=True)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 42 kB 498 kB/s 
     |████████████████████████████████| 806 kB 6.8 MB/s 
     |████████████████████████████████| 8.0 MB 20.1 MB/s 
     |████████████████████████████████| 2.8 MB 24.2 MB/s 
     |████████████████████████████████| 398 kB 43.5 MB/s 
     |████████████████████████████████| 57 kB 6.3 MB/s 
     |████████████████████████████████| 140 kB 57.1 MB/s 
     |████████████████████████████████| 636 kB 42.0 MB/s 
     |████████████████████████████████| 829 kB 45.5 MB/s 
     |██████████████████████████████▎ | 834.1 MB 1.2 MB/s eta 0:00:40tcmalloc: large alloc 1147494400 bytes == 0x3736000 @  0x7f4ca7435615 0x592b76 0x4df71e 0x59afff 0x515655 0x549576 0x593fce 0x548ae9 0x51566f 0x549576 0x593fce 0x548ae9 0x5127f1 0x598e3b 0x511f68 0x598e3b 0x511f68 0x598e3b 0x511f68 0x4bc98a 0x532e76 0x594b72 0x515600 0x549576 0x593fce 0x548ae9 0x5127f1 0x549576 0x

In [2]:
#모델 환경 설정
import torch
from ratsnlp.nlpbook.generation import GenerationTrainArguments
args = GenerationTrainArguments(
    pretrained_model_name="skt/kogpt2-base-v2",
    downstream_corpus_name="nsmc",
    downstream_model_dir="/gdrive/My Drive/nlpbook/checkpoint-generation",
    max_seq_length=32,
    batch_size=32 if torch.cuda.is_available() else 4,
    learning_rate=5e-5,
    epochs=3,
    tpu_cores=0 if torch.cuda.is_available() else 8,
    seed=7,
)

/usr/local/lib/python3.7/dist-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: libtorch_cuda_cu.so: cannot open shared object file: No such file or directory
  warn(f"Failed to load image Python extension: {e}")


In [3]:
#랜덤 시드 고정
from ratsnlp import nlpbook
nlpbook.set_seed(args)

set seed: 7


In [4]:
#로거 설정
nlpbook.set_logger(args)

INFO:ratsnlp:Training/evaluation parameters GenerationTrainArguments(pretrained_model_name='skt/kogpt2-base-v2', downstream_task_name='sentence-generation', downstream_corpus_name='nsmc', downstream_corpus_root_dir='/content/Korpora', downstream_model_dir='/gdrive/My Drive/nlpbook/checkpoint-generation', max_seq_length=32, save_top_k=1, monitor='min val_loss', seed=7, overwrite_cache=False, force_download=False, test_mode=False, learning_rate=5e-05, epochs=3, batch_size=32, cpu_workers=4, fp16=False, tpu_cores=0)


#### 말뭉치 다운로드

In [5]:
#말뭉치 다운로드
from Korpora import Korpora
Korpora.fetch(
    corpus_name=args.downstream_corpus_name,
    root_dir=args.downstream_corpus_root_dir,
    force_download=args.force_download,
)

[nsmc] download ratings_train.txt: 14.6MB [00:00, 73.2MB/s]                           
[nsmc] download ratings_test.txt: 4.90MB [00:00, 33.7MB/s]                           


#### 토크나이저 준비하기

In [6]:
#토크나이저 준비하기
from transformers import PreTrainedTokenizerFast
tokenizer = PreTrainedTokenizerFast.from_pretrained(
    args.pretrained_model_name,
    eos_token="</s>",
    #eos_token은 문장 마지막에 붙이는 스페셜 토큰, sk텔레콤이 메델을 프리트레인할 때 지정했기에 같은 방식으로 사용 
)

Downloading:   0%|          | 0.00/2.83M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.00k [00:00<?, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.


#### 데이터 전처리하기

In [7]:
from ratsnlp.nlpbook.generation import NsmcCorpus, GenerationDataset
from torch.utils.data import DataLoader, SequentialSampler, RandomSampler
corpus = NsmcCorpus()
train_dataset = GenerationDataset(
    args=args,
    corpus=corpus,
    tokenizer=tokenizer,
    mode="train",
)
train_dataloader = DataLoader(
    train_dataset,
    batch_size=args.batch_size,
    sampler=RandomSampler(train_dataset, replacement=False),
    collate_fn=nlpbook.data_collator,
    drop_last=False,
    num_workers=args.cpu_workers,
)

INFO:ratsnlp:Creating features from dataset file at /content/Korpora/nsmc
INFO:ratsnlp:loading train data... LOOKING AT /content/Korpora/nsmc/ratings_train.txt
INFO:ratsnlp:tokenize sentences, it could take a lot of time...
INFO:ratsnlp:tokenize sentences [took 6.503 s]
INFO:ratsnlp:*** Example ***
INFO:ratsnlp:sentence: 부정 아 더빙.. 진짜 짜증나네요 목소리
INFO:ratsnlp:tokens: ▁부정 ▁아 ▁더 빙 .. ▁진짜 ▁짜 증 나 네 요 ▁목소리 </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s>
INFO:ratsnlp:features: GenerationFeatures(input_ids=[11775, 9050, 9267, 7700, 9705, 23971, 12870, 8262, 7055, 7098, 8084, 48213, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], attention_mask=[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], token_type_ids=[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], labels=[11775, 9050, 9267, 7700, 9705, 23971, 12870, 8262, 7055, 7098, 8084, 48213

#### 테스트 데이터 구축

In [8]:
val_dataset = GenerationDataset(
    args=args,
    corpus=corpus,
    tokenizer=tokenizer,
    mode="test",
)
val_dataloader = DataLoader(
    val_dataset,
    batch_size=args.batch_size,
    sampler=SequentialSampler(val_dataset),
    collate_fn=nlpbook.data_collator,
    drop_last=False,
    num_workers=args.cpu_workers,
)

INFO:ratsnlp:Creating features from dataset file at /content/Korpora/nsmc
INFO:ratsnlp:loading test data... LOOKING AT /content/Korpora/nsmc/ratings_test.txt
INFO:ratsnlp:tokenize sentences, it could take a lot of time...
INFO:ratsnlp:tokenize sentences [took 2.313 s]
INFO:ratsnlp:*** Example ***
INFO:ratsnlp:sentence: 긍정 굳 ㅋ
INFO:ratsnlp:tokens: ▁긍정 ▁굳 ▁ ᄏ </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s>
INFO:ratsnlp:features: GenerationFeatures(input_ids=[16420, 12969, 739, 605, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], attention_mask=[1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], token_type_ids=[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], labels=[16420, 12969, 739, 605, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,

#### 프리트레인 마친 모델 읽어들이기

In [9]:
from transformers import GPT2LMHeadModel
#GPT2LMHeadModelsms KoGPT2가 프리트렌할 때 썼던 모델클래스 
#GPT2는 프리트레인 태스크와 파인튜닝태스크가 다음 단어 맞히기로 같기에 똑같은 모델 클래스를 사용
model = GPT2LMHeadModel.from_pretrained(
    args.pretrained_model_name
)

Downloading:   0%|          | 0.00/513M [00:00<?, ?B/s]

#### 모델 파인튜닝하기

파이토치 라이트닝이 제공하는 lightning 모듈을 상속받아 모델과 옵티마이저, 학습 과정 등이 정의된 테스크를 정의

옵티마이저: 아담

In [10]:
from ratsnlp.nlpbook.generation import GenerationTask
task = GenerationTask(model, args)

In [11]:
trainer = nlpbook.get_trainer(args)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores


#### 학습하기

In [12]:
trainer.fit(
    task,
    train_dataloader=train_dataloader,
    val_dataloaders=val_dataloader,
)
#학습 결과물(체크포인트)은 미리 연동해둔 구글 드라이브의 준비된 위치(/gdrive/My Drive/nlpbook/checkpoint-generation)에 저장

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type            | Params
------------------------------------------
0 | model | GPT2LMHeadModel | 125 M 
------------------------------------------
125 M     Trainable params
0         Non-trainable params
125 M     Total params
500.656   Total estimated model params size (MB)


Training: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

## 프리트레인 마친 모델로 문장 생성하기

In [ ]:
# !pip install ratsnlp

### 모델 로딩

프리트레인한 GPT2 모델과 토크나이저 읽기

In [13]:
from transformers import GPT2LMHeadModel
model = GPT2LMHeadModel.from_pretrained(
    "skt/kogpt2-base-v2",
)
model.eval()

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(51200, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0): GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
      (1): GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )


In [14]:
from transformers import PreTrainedTokenizerFast
tokenizer = PreTrainedTokenizerFast.from_pretrained(
    "skt/kogpt2-base-v2",
    eos_token="</s>",
)

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.


### 프롬프트 준비

In [15]:
input_ids = tokenizer.encode("안녕하세요", return_tensors="pt")

### Greedy Search

다음 단어 확률 분포에서 최대 확률을 내는 단어들을 리턴한다.

여러 번 수행하더라도 생성 결과가 바뀌지 않는다(do_sample=False).

max_length는 생성 최대 길이이며 이보다 길거나, 짧더라도 EOD 토큰 등 스페셜 토큰이 나타나면 생성을 중단한다.

min_length는 생성 최소 길이이며 이보다 짧은 구간에서 EOD 등 스페셜 토큰이 등장해 생성이 중단될 경우 해당 토큰이 나올 확률을 0으로 수정하여 문장 생성이 종료되지 않도록 강제한다.

In [16]:
import torch
with torch.no_grad():
    generated_ids = model.generate(
        input_ids,
        do_sample=False,#확률값이 높은 단어를 다음 단어로 결정
        min_length=10,
        max_length=50,
    )
    print(tokenizer.decode([el.item() for el in generated_ids[0]]))

안녕하세요?"
"그럼, 그건 뭐예요?"
"그럼, 그건 뭐예요?"
"그럼, 그건 뭐예요?"
"그럼, 그건 뭐예요?"



### Beam Search 



Beam Search는 다음 단어 확률 분포에서 `num_beams`만큼의 경우의 수를 남겨가면서 문장을 생성한다. 

Beam search는 Greedy search보다 계산량이 많지만 좀 더 확률값이 높은 문장을 생성할 수 있다.

In [17]:
with torch.no_grad():
    generated_ids = model.generate(
        input_ids,
        do_sample=False,
        min_length=10,
        max_length=50,
        num_beams=3,#빔 크기를 3으로 지정
    )
    print(tokenizer.decode([el.item() for el in generated_ids[0]]))

안녕하세요?"
"그렇지 않습니다."
"그렇지 않습니다."
"그렇지 않습니다."
"그렇지 않습니다."
"그렇지 않습니다."
"그렇지 않습니다."
"그


`num_beams=1`로 설정하면 정확히 Greedy search와 동일하게 작동

In [18]:
with torch.no_grad():
    generated_ids = model.generate(
        input_ids,
        do_sample=False,
        min_length=10,
        max_length=50,
        num_beams=1,
    )
    print(tokenizer.decode([el.item() for el in generated_ids[0]]))

안녕하세요?"
"그럼, 그건 뭐예요?"
"그럼, 그건 뭐예요?"
"그럼, 그건 뭐예요?"
"그럼, 그건 뭐예요?"



### 반복 줄이기

#### 반복되는 n-gram 사이즈를 지정하기
 3개 이상의 토큰이 반복될 경우 해당 3-gram 등장 확률을 0으로 만들어 생성 결과에서 배제합니다.

In [19]:
with torch.no_grad():
    generated_ids = model.generate(
        input_ids,
        do_sample=False,
        min_length=10,
        max_length=50,
        no_repeat_ngram_size=3,#토큰 3개 이상 반복될 경우 3번째 토큰 확률 0으로 변경
    )
    print(tokenizer.decode([el.item() for el in generated_ids[0]]))

안녕하세요?"
"그럼, 그건 뭐예요?" 하고 나는 물었다.
"그건 뭐죠?" 나는 물었다.
나는 대답하지 않았다.
"그런데 왜 그걸 물어요? 그건 무슨 뜻이에요?


#### repetition penalty

repetition penalty로 반복을 통제할 수도 

범위는 1 이상의 값을 가져야 한다. 
1이라면 아무런 패널티를 적용하지 않는게 된다.

In [20]:
with torch.no_grad():
    generated_ids = model.generate(
        input_ids,
        do_sample=False,
        min_length=10,
        max_length=50,
        repetition_penalty=1.0,#리피트션 패널티 적용
    )
    print(tokenizer.decode([el.item() for el in generated_ids[0]]))

안녕하세요?"
"그럼, 그건 뭐예요?"
"그럼, 그건 뭐예요?"
"그럼, 그건 뭐예요?"
"그럼, 그건 뭐예요?"



repetition_penalty 값이 클 수록 패널티가 세게 적용

In [21]:
with torch.no_grad():
    generated_ids = model.generate(
        input_ids,
        do_sample=False,
        min_length=10,
        max_length=50,
        repetition_penalty=1.1,
    )
    print(tokenizer.decode([el.item() for el in generated_ids[0]]))

안녕하세요?"
"그럼, 그건 뭐예요?"
"아니요, 저는요."
"그럼, 그건 무슨 말씀이신지요?"
"그럼, 그건 뭐예요?"



In [22]:
with torch.no_grad():
    generated_ids = model.generate(
        input_ids,
        do_sample=False,
        min_length=10,
        max_length=50,
        repetition_penalty=1.5,
    )
    print(tokenizer.decode([el.item() for el in generated_ids[0]]))

안녕하세요?"
"그럼, 그건 뭐예요, 아저씨. 저는 지금 이 순간에도 괜찮아요. 그리고 제가 할 수 있는 일은 아무것도 없어요.
이제 그만 돌아가고 싶어요.
제가 하는 일이 무엇


### top-k sampling

지금까지는 생성을 반복하더라도 그 결과가 동일한 샘플링 방식이다.

**top-k sampling**은 다음 단어를 뽑을 때 확률값 기준 가장 큰 k개 가운데 하나를 선택하는 기법다. 
확률값이 큰 단어가 다음 단어로 뽑힐 가능성이 높아지지만, k개 안에 있는 단어라면 확률값이 낮더라도 다음 단어로 추출될 수 있다. 따라서 top-k sampling은 **매 시행 때마다 생성 결과가 달라진다.** k는 1 이상의 값을 지녀야 한다.

In [23]:
with torch.no_grad():
    generated_ids = model.generate(
        input_ids,
        do_sample=True,#샘플링 방식으로 다음 토큰 생성
        min_length=10,
        max_length=50,
        top_k=50,#k를 50으로 탑 k 샘플링 수행
    )
    print(tokenizer.decode([el.item() for el in generated_ids[0]]))

안녕하세요!???????
그럼, 당신이 이번에 다시 한번만 더 좋은 소식이 될 것 같네요...
오늘 저녁 6시까지 오셨어요!!~
오늘은 아침부터 오늘


k=1일 경우 Greedy search와 동일

In [24]:
with torch.no_grad():
    generated_ids = model.generate(
        input_ids,
        do_sample=True,
        min_length=10,
        max_length=50,
        top_k=1,
    )
    print(tokenizer.decode([el.item() for el in generated_ids[0]]))

안녕하세요?"
"그럼, 그건 뭐예요?"
"그럼, 그건 뭐예요?"
"그럼, 그건 뭐예요?"
"그럼, 그건 뭐예요?"



#### top-k sampling + temperature scaling
top-k sampling은 temperature scaling과 동시에 적용할 수

(1) t가 0에 가까워질 수록 토큰 분포가 sharp해진다 > 1등 토큰이 뽑힐 확률이 그만큼 높아진다 > do_sample=True이지만 사실상 greedy decoding이 된다

In [25]:
with torch.no_grad():
    generated_ids = model.generate(
        input_ids,
        do_sample=True,
        min_length=10,
        max_length=50,
        top_k=50,
        temperature=0.01, #탬퍼러처 스케일링 적용
    )
    print(tokenizer.decode([el.item() for el in generated_ids[0]]))

안녕하세요?"
"그럼, 그건 뭐예요?"
"그럼, 그건 뭐예요?"
"그럼, 그건 뭐예요?"
"그럼, 그건 뭐예요?"



(2) t=1이라면 모델 출력 분포를 그대로 사용한다 > 하지만 샘플링 방식을 사용하기 때문에 생성할 때마다 다른 문장이 나온다

In [26]:
with torch.no_grad():
    generated_ids = model.generate(
        input_ids,
        do_sample=True,
        min_length=10,
        max_length=50,
        top_k=50,
        temperature=1.0,
    )
    print(tokenizer.decode([el.item() for el in generated_ids[0]]))

안녕하세요?"
"예, 예. 저는, 고마워요. 내일 오후에 만나죠."
"예, 예, 고마워요."
내가 말끝을 흐렸다.
"아무쪼록, 저도 이만 가볼


(3) t를 키울수록 토큰 분포가 uniform해진다 > 사실상 uniform sampling이 된다, 생성 품질이 악화할 가능성이 높아진다

In [27]:
with torch.no_grad():
    generated_ids = model.generate(
        input_ids,
        do_sample=True,
        min_length=10,
        max_length=50,
        top_k=50,
        temperature=100000000.0,
    )
    print(tokenizer.decode([el.item() for el in generated_ids[0]]))

안녕하세요라고 하면 아플 일이죠.
자 제가 이렇게 궁금해지는 쵸 네 사실 네 그 요즘 너무 막 제가 안 해 볼 생각이 돼 가지신 아 쩜 안 좋다가 요러지고 어르케 네 한 이백이백 


### top-p sampling


top-p sampling은 다음 단어를 뽑을 때 **누적 확률값이 p 이하인 단어들 가운데 하나를 선택**하는 기법이다. 

확률값이 큰 단어가 다음 단어로 뽑힐 가능성이 높아지지만, 누적 확률값 p 이하에 있는 단어라면 확률값이 낮더라도 다음 단어로 추출될 수 있다. 따라서 top-p sampling은 **매 시행 때마다 생성 결과가 달라진다.** 

p는 확률이기 때문에 0~1 사이의 값을 지녀야 합니다. p가 1이라면 어휘 집합에 있는 모든 단어를 대상으로 샘플링하기 때문에 top-p sampling 효과가 사라진다. p가 1보다 약간 작다면 확률값이 낮은 일부 단어들을 다음 단어 후보에서 제거해 생성 품질을 높인다. 

In [28]:
with torch.no_grad():
    generated_ids = model.generate(
        input_ids,
        do_sample=True,
        min_length=10,
        max_length=50,
        top_p=0.92,
    )
    print(tokenizer.decode([el.item() for el in generated_ids[0]]))

안녕하세요?"
"네! 저는 이렇게 뵙겠습니다."
"그래서 이렇게 하죠."
오가는 부인에게 말을 걸었다.
"내가 이렇게 해주었으니 얼마나 다행인가. 정말."
"그렇다면 우리도 기꺼이


p가 0에 가까울 경우 Greedy search와 비슷해 진다.

In [29]:
with torch.no_grad():
    generated_ids = model.generate(
        input_ids,
        do_sample=True,
        min_length=10,
        max_length=50,
        top_p=0.01,
    )
    print(tokenizer.decode([el.item() for el in generated_ids[0]]))

안녕하세요?"
"그럼, 그건 뭐예요?"
"그럼, 그건 뭐예요?"
"그럼, 그건 뭐예요?"
"그럼, 그건 뭐예요?"



### 통합 적용


허깅페이스(huggingface) 라이브러리의 구현상 적용 순서
- _get_logits_processor
  - RepetitionPenalty
  - NoRepeatNGramLogits
  - MinLengthLogits
- _get_logits_warper
  - TemperatureLogits
  - TopKLogits
  - TopPLogits

In [30]:
with torch.no_grad():
    generated_ids = model.generate(
        input_ids,
        do_sample=True,#샘플링 방식 적용
        min_length=10,#문장길이 설정
        max_length=50,
        repetition_penalty=1.5,#반복 줄이기
        no_repeat_ngram_size=3,
        temperature=0.9,#탬퍼리처 스케일링
        top_k=50,
        top_p=0.92,
    )
    print(tokenizer.decode([el.item() for el in generated_ids[0]]))

안녕하세요?' 하고 반문한 뒤 다시 그 자리에 주저앉았다.
이제는 아무 생각 없이 말을 이었다.
"미안해요. 하지만 저는 우리 당신들이 잘못했어요, 제발!"
나는 고개를 끄덕였다.
하지만


## 파인튜닝 마친 모델로 문장 생성

### 설정하기

In [ ]:
#!pip install ratsnlp

# from google.colab import drive
# drive.mount('/gdrive', force_remount=True)

In [31]:
from ratsnlp.nlpbook.generation import GenerationDeployArguments
args = GenerationDeployArguments(
    pretrained_model_name="skt/kogpt2-base-v2",
    downstream_model_dir="/gdrive/My Drive/nlpbook/checkpoint-generation",
)

downstream_model_checkpoint_fpath: /gdrive/My Drive/nlpbook/checkpoint-generation/epoch=1-val_loss=2.29.ckpt


### 모델 로딩

In [32]:
import torch
from transformers import GPT2Config, GPT2LMHeadModel
pretrained_model_config = GPT2Config.from_pretrained(
    args.pretrained_model_name,
)
model = GPT2LMHeadModel(pretrained_model_config)
fine_tuned_model_ckpt = torch.load(
    args.downstream_model_checkpoint_fpath,
    map_location=torch.device("cpu"),
)
model.load_state_dict({k.replace("model.", ""): v for k, v in fine_tuned_model_ckpt['state_dict'].items()})
model.eval()

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(51200, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0): GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
      (1): GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )


In [33]:
from transformers import PreTrainedTokenizerFast
tokenizer = PreTrainedTokenizerFast.from_pretrained(
    args.pretrained_model_name,
    eos_token="</s>",
)

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.


### 인퍼런스 함수 선언

In [34]:
def inference_fn(
        prompt,
        min_length=10,
        max_length=20,
        top_p=1.0,
        top_k=50,
        repetition_penalty=1.0,
        no_repeat_ngram_size=0,
        temperature=1.0,
):
    try:
        input_ids = tokenizer.encode(prompt, return_tensors="pt")
        with torch.no_grad():
            generated_ids = model.generate(
                input_ids,
                do_sample=True,
                top_p=float(top_p),
                top_k=int(top_k),
                min_length=int(min_length),
                max_length=int(max_length),
                repetition_penalty=float(repetition_penalty),
                no_repeat_ngram_size=int(no_repeat_ngram_size),
                temperature=float(temperature),
           )
        generated_sentence = tokenizer.decode([el.item() for el in generated_ids[0]])
    except:
        generated_sentence = """처리 중 오류가 발생했습니다. <br>
            변수의 입력 범위를 확인하세요. <br><br> 
            min_length: 1 이상의 정수 <br>
            max_length: 1 이상의 정수 <br>
            top-p: 0 이상 1 이하의 실수 <br>
            top-k: 1 이상의 정수 <br>
            repetition_penalty: 1 이상의 실수 <br>
            no_repeat_ngram_size: 1 이상의 정수 <br>
            temperature: 0 이상의 실수
            """
    return {
        'result': generated_sentence,
    }

In [36]:
inference_fn('이 영화는 정말 똥이다.')

{'result': '이 영화는 정말 똥이다...ᄃᄃᄃᄃ</s>'}